In [87]:
import numpy as np
from qiskit.opflow import I, Z
from qiskit.aqua.operators import SummedOp, PauliOp, MatrixOp
from qiskit.quantum_info import Pauli

In [2]:
N = 5
num_turns = 2*(N - 1)
side_chain = [0]*N
lambda_back = 10

# GLOBAL
FULL_ID = I
for i in range(1, num_turns):
    FULL_ID = I^FULL_ID

In [3]:
def _create_pauli_for_conf(N):
    terms = []
    terms.reverse()
    num_turns = 2*(N - 1)
    pauli_conf = np.zeros((num_turns, 2), dtype=object)

    for index in range(num_turns):
        if index != 0: 
            temp = I
        else: 
            temp = Z
        for i in range(1, num_turns):
            if i == index: 
                temp = temp^Z
            else:
                temp = temp^I
        terms.append(temp)
#     terms.reverse()
    
    for i in range(num_turns):
        pauli_conf[i][0] = terms[i]
        pauli_conf[i][1] = terms[i]   
    return pauli_conf

def _create_qubits_for_conf(pauli_conf):
    qubits = np.zeros(pauli_conf.shape, dtype=object)
    num_turns = qubits.shape[0]
    for i in range(num_turns):
        qubits[i][0] = (0.5*FULL_ID - 0.5*pauli_conf[i][0])
        qubits[i][1] = (0.5*FULL_ID - 0.5*pauli_conf[i][1])
    return qubits

# Create paulis for conformation

In [4]:
pauli_conf = _create_pauli_for_conf(N)

In [5]:
pauli_conf

array([[PauliOp(Pauli('ZIIIIIII'), coeff=1.0),
        PauliOp(Pauli('ZIIIIIII'), coeff=1.0)],
       [PauliOp(Pauli('IZIIIIII'), coeff=1.0),
        PauliOp(Pauli('IZIIIIII'), coeff=1.0)],
       [PauliOp(Pauli('IIZIIIII'), coeff=1.0),
        PauliOp(Pauli('IIZIIIII'), coeff=1.0)],
       [PauliOp(Pauli('IIIZIIII'), coeff=1.0),
        PauliOp(Pauli('IIIZIIII'), coeff=1.0)],
       [PauliOp(Pauli('IIIIZIII'), coeff=1.0),
        PauliOp(Pauli('IIIIZIII'), coeff=1.0)],
       [PauliOp(Pauli('IIIIIZII'), coeff=1.0),
        PauliOp(Pauli('IIIIIZII'), coeff=1.0)],
       [PauliOp(Pauli('IIIIIIZI'), coeff=1.0),
        PauliOp(Pauli('IIIIIIZI'), coeff=1.0)],
       [PauliOp(Pauli('IIIIIIIZ'), coeff=1.0),
        PauliOp(Pauli('IIIIIIIZ'), coeff=1.0)]], dtype=object)

# Create qubits for conformation

In [6]:
qubits = _create_qubits_for_conf(pauli_conf)

In [25]:
def _create_indic_turn(N, side_chain, qubits):
    if len(side_chain)!= N:
        raise Exception('size of side_chain list is not equal to N')
    num_turns = N - 1 
    indic_0 = np.zeros((num_turns, 2), dtype=object)
    indic_1 = np.zeros((num_turns, 2), dtype=object)
    indic_2 = np.zeros((num_turns, 2), dtype=object)
    indic_3 = np.zeros((num_turns, 2), dtype=object)
    r_conf = 0
    for i in range(num_turns):
        for m in range(2):
            if m == 1:
                if side_chain[i - 1] == 0:
                    continue
                else:
                    pass
            indic_0[i][m] = (FULL_ID - qubits[2*i - 1][m])^(FULL_ID - qubits[2*i][m])
            indic_1[i][m] = qubits[2*i][m]^(qubits[2*i][m] - 1*qubits[2*i - 1][m])
            indic_2[i][m] = qubits[2*i - 1][m]^(qubits[2*i - 1][m] -1*qubits[2*i][m])
            indic_3[i][m] = qubits[2*i - 1][m]^qubits[2*i][m]
            indic_0[i][m] = indic_0[i][m].reduce()
            indic_1[i][m] = indic_1[i][m].reduce()
            indic_2[i][m] = indic_2[i][m].reduce()
            indic_3[i][m] = indic_3[i][m].reduce()
            r_conf += 1
    num_qubits = 2*r_conf - 5
    print('number of qubits required for conformation: ', num_qubits)
    return indic_0, indic_1, indic_2, indic_3, num_qubits


def _check_turns(i, p, j, s,
                 indic0, indic1, indic2,
                 indic3, pauli_conf):
    t_ij = indic0[i][p]^indic0[j][s]
    t_ij.add(indic1[i][p]^indic1[j][s])
    t_ij.add(indic2[i][p]^indic2[j][s])
    t_ij.add(indic3[i][p]^indic3[j][s]) 
    t_ij = t_ij.reduce()
    return t_ij


def _create_H_back(N, lambda_back, indic_0,
                   indic_1, indic_2, indic_3,
                   pauli_conf):
    H_back = 0
    for i in range(N - 2):
        H_back += lambda_back*_check_turns(i, 0, i + 1, 0,
                                           indic_0, indic_1, indic_2, indic_3, pauli_conf)
    return H_back

In [26]:
indic_0, indic_1, indic_2, indic_3, num_qubits = _create_indic_turn(N, side_chain, qubits)
H_back = _create_H_back(N, lambda_back, indic_0, indic_1, indic_2, indic_3, pauli_conf)

number of qubits required for conformation:  3


In [27]:
for i in indic_3:
    print('term is: {}'.format(i[0]))

term is: 0.25 * IIIIIIIIIIIIIIII
- 0.25 * IIIIIIIZIIIIIIII
- 0.25 * IIIIIIIIZIIIIIII
+ 0.25 * IIIIIIIZZIIIIIII
term is: 0.25 * IIIIIIIIIIIIIIII
- 0.25 * IZIIIIIIIIIIIIII
- 0.25 * IIIIIIIIIIZIIIII
+ 0.25 * IZIIIIIIIIZIIIII
term is: 0.25 * IIIIIIIIIIIIIIII
- 0.25 * IIIZIIIIIIIIIIII
- 0.25 * IIIIIIIIIIIIZIII
+ 0.25 * IIIZIIIIIIIIZIII
term is: 0.25 * IIIIIIIIIIIIIIII
- 0.25 * IIIIIZIIIIIIIIII
- 0.25 * IIIIIIIIIIIIIIZI
+ 0.25 * IIIIIZIIIIIIIIZI


In [101]:
print(indic_3[0][0])

0.25 * IIIIIIIIIIIIIIII
- 0.25 * IIIIIIIZIIIIIIII
- 0.25 * IIIIIIIIZIIIIIII
+ 0.25 * IIIIIIIZZIIIIIII


In [118]:
print(type(indic_3[0][0].primitive[0].table.array))

<class 'numpy.ndarray'>
